https://adventofcode.com/2020/day/11

In [1]:
file = '2020-12-11_input.txt'  #2020-12-11_input.txt
with open(file) as f:
    records = f.read().split('\n')
    
records = [x for x in records[:-1]]
len(records)

98

## First part

In [2]:
import numpy as np
def split(line): 
    return [char for char in line]  
a = np.array([split(x) for x in records])
a = np.where(a=='.', np.nan, a)
a = np.where(a=='L', 0, a)
a = a.astype(np.float32)
a
b = np.zeros([a.shape[0],a.shape[1],2])
b[:,:,0] = a

In [3]:
def assess(x,y,z):
    x1 = max(x-1,0)
    x2 = min(x+2,a.shape[0])
    y1 = max(y-1,0)
    y2 = min(y+2,a.shape[1])
    window = b[x1:x2,y1:y2,z]
    #print(window)
    
    if np.isnan(b[x,y,z]):
        result = np.nan
    elif b[x,y,z] == 0:
        result = int(np.nansum(window) < 1) 
    elif b[x,y,z] == 1:
        result = int(np.nansum(window) < 5)
        
    return result


In [4]:
it_changes = True
c = 0
seated = 0
while it_changes:
    zs = c%2 # source
    c+=1
    zt = abs(zs-1)
    for x in range(b.shape[0]):
        for y in range(b.shape[1]):
            b[x,y,zt] = assess(x,y,zs)
    prev_seated = seated
    seated = np.nansum(b[:,:,zt])
    print(seated)
    if prev_seated == seated:
        print('stabilized')
        it_changes = False

7564.0
180.0
7071.0
378.0
6544.0
558.0
6102.0
735.0
5729.0
891.0
5389.0
1051.0
5045.0
1199.0
4715.0
1327.0
4430.0
1454.0
4169.0
1571.0
3933.0
1684.0
3721.0
1792.0
3502.0
1900.0
3301.0
2001.0
3139.0
2084.0
2993.0
2156.0
2877.0
2211.0
2811.0
2244.0
2740.0
2278.0
2680.0
2299.0
2644.0
2317.0
2611.0
2328.0
2581.0
2346.0
2551.0
2358.0
2531.0
2368.0
2516.0
2373.0
2502.0
2377.0
2491.0
2380.0
2482.0
2383.0
2474.0
2388.0
2463.0
2395.0
2448.0
2403.0
2436.0
2409.0
2427.0
2414.0
2424.0
2418.0
2418.0
stabilized


## Second part

Idea: 
 1. Flatten out
 2. Calculate index of all visible fields based on compass walk (like a stendcil)
 3. Iterate until converging

### Flattening out

In [5]:
file = '2020-12-11_input.txt'  #2020-12-11_input.txt
with open(file) as f:
    records = f.read().split('\n')
    records = records[:-1] # cut empty line
len_x = len(records)
len_y = len(records[0])
flat_records = ''.join(records)
print(len_x,len_y)

98 96


### Compass rules

In [6]:
def north(p,x,y):
    while (p-x) > 0: 
        # as long as we are not in the first row
        p = p-x
        if val[p] >= 0:
            return p

def north_east(p,x,y):
    while (((p-x) > 0) and ((p+1)%x > 0)):
        p = p-x+1
        if val[p] >= 0:
            return p

def east(p,x,y): 
    while ((p+1)%x > 0): 
        # as long as we are not in the last column
        p = p+1
        try:
            if val[p] >= 0:
                return p
        except:
            print('failed east',p)

def south_east(p,x,y):
    while (((p+x) <= x*y-1) and ((p+1)%x > 0)):
        p = p+x+1
        if val[p] >= 0:
            return p
    
def south(p,x,y):
    while (p+x) <= x*y-1: 
        # as long as we are not in the last row
        p = p+x
        if val[p] >= 0:
            return p

def south_west(p,x,y):
    while (((p+x) <= x*y-1) and ((p%x) > 0)):
        p = p+x-1
        if val[p] >= 0:
            return p

def west(p,x,y):
    while ((p%x) > 0): 
        # as long as we are not in the first column
        p = p-1
        if val[p] >= 0:
            return p

def north_west(p,x,y):
    while (((p-x) > 0) and ((p%x) > 0)):
        p = p-x-1
        if val[p] >= 0:
            return p


### Compass walk
Building up ref[] with list of indices each position can "see".

In [7]:
val = []
for i in flat_records:
    if i == 'L':
        val.append(0)
    if i == '.':
        val.append(-1)
ref = []
x = len_x
y = len_y
for i in range(len(val)):
    
    tmp = []
    tmp.append(north(i,x,y))
    tmp.append(north_east(i,x,y))
    #try:
    tmp.append(east(i,x,y))
    #except:
    #    print('broken:',i)
        #tmp.append(east(i,x,y))
    tmp.append(south_east(i,x,y))
    tmp.append(south(i,x,y))
    tmp.append(south_west(i,x,y))
    tmp.append(west(i,x,y))
    tmp.append(north_west(i,x,y))
    ref.append([x for x in tmp if x is not None])

In [8]:
ref[0:5]

[[1, 99, 98],
 [2, 100, 99, 98, 0],
 [3, 101, 100, 99, 1],
 [4, 102, 101, 100, 2],
 [5, 103, 102, 101, 3]]

In [9]:
seated = 0
moving = True
while moving:
    prev_seated = seated

    tmp = val.copy()
    for j in range(len(val)):
        if (val[j] == 1):
            if (sum([val[i] for i in ref[j]]) >= 5):
                tmp[j] = 0
        elif (val[j] == 0):
            if (sum([val[i] for i in ref[j]]) == 0):
                tmp[j] = 1
    if val == tmp:
        moving=False
        seated = sum([i for i in tmp if i >= 0])
        print('DONE', seated)
    val = tmp.copy()
    seated = sum([i for i in val if i >= 0])
    print('seated:',seated)


seated: 7564
seated: 5
seated: 7551
seated: 13
seated: 7527
seated: 25
seated: 7490
seated: 40
seated: 7443
seated: 59
seated: 7387
seated: 81
seated: 7323
seated: 107
seated: 7247
seated: 144
seated: 7152
seated: 186
seated: 7040
seated: 231
seated: 6915
seated: 285
seated: 6770
seated: 335
seated: 6638
seated: 391
seated: 6488
seated: 459
seated: 6323
seated: 540
seated: 6133
seated: 601
seated: 5960
seated: 681
seated: 5752
seated: 767
seated: 5521
seated: 858
seated: 5280
seated: 945
seated: 5054
seated: 1030
seated: 4824
seated: 1126
seated: 4594
seated: 1215
seated: 4376
seated: 1305
seated: 4172
seated: 1383
seated: 3976
seated: 1459
seated: 3776
seated: 1534
seated: 3588
seated: 1612
seated: 3403
seated: 1684
seated: 3251
seated: 1748
seated: 3108
seated: 1798
seated: 2978
seated: 1848
seated: 2854
seated: 1897
seated: 2733
seated: 1938
seated: 2626
seated: 1976
seated: 2536
seated: 2006
seated: 2452
seated: 2038
seated: 2383
seated: 2063
seated: 2322
seated: 2089
seated: 2261


In [ ]:
2144